In [7]:
import numpy as np
import h5py
import matplotlib
matplotlib.rcParams["text.usetex"]="true"
import matplotlib.pyplot as plt
import matplotlib.colors as colors

subgroup=dict(tw=0,Delta=5,gamma=0.05)


with h5py.File("data.hdf5","r") as IO:
    for grp in IO:
        pars = eval(grp.split("_")[-1])
        if all(pars[k]==subgroup[k] for k in subgroup):
            L = pars["L"]
            gamma = pars["gamma"]
            T = 2*np.pi/pars["Omega"]
            print((pars["tp"]+pars["tw"])*T)
            
            k = np.arange(-L//2+1,L//2+1,1)*2.0/L
            
            V_list = []
            n_list = []
            for dataset in IO[grp]:
                
                if dataset=="Ef":
                    continue
                V = float(dataset.split("_")[-1])
                n = IO[grp][dataset][2:,:].sum(axis=0)
                V_list.append(V)
                n_list.append(n)
            
            Omega,Delta = pars["Omega"],pars["Delta"]


            V_list = np.asarray(V_list)
            n_list = np.asarray(n_list)
            
            i = np.argsort(V_list)
            V_list = V_list[i]
            n_list = n_list[i]
            
#             m = np.ones_like(V_list,dtype=np.bool)
            m = np.argwhere(np.abs(V_list)<15).ravel()
            r = 2*k.size/V_list[m].size

            f,ax = plt.subplots(1,1,figsize=(3.5,7))
            try:
                Ef = np.asarray(IO[grp]["Ef"])
                ax.plot(k,Ef,marker="",color="red",linestyle=":")
            except: 
                pass
#             ax.plot(k,Ef+Omega,marker="",color="red",linestyle=":")
#             ax.plot(k,Ef-Omega,marker="",color="red",linestyle=":")
#             ax.plot(k,Ef+2*Omega,marker="",color="red",linestyle=":")
#             ax.plot(k,Ef-2*Omega,marker="",color="red",linestyle=":")

            vmax = n_list[m,:].max()
            vmin = 1e-4
            mesh = ax.pcolormesh(k,V_list[m],n_list[m,:],shading='auto',norm=colors.LogNorm(vmin,vmax))
            ax.set_aspect(0.25)
            ax.set_xticks([-1,0,1])
            ax.set_ylabel("$V_g$",fontsize=12)
            
            plt.xlabel("$k/\pi$",fontsize=12)
            
            plt.colorbar(mesh,ax=ax)
            
            pars = "_".join(str(k)+"_"+str(v) for k,v in pars.items())
            plt.savefig("plots/spectrum_{}.jpg".format(pars),bbox_inches="tight",dpi=500)
            plt.clf()
                
#             plt.plot(V_list[m],n_list[m,:].sum(axis=1))
#             plt.show()                
                
    
    

15.079644737231007


<Figure size 252x504 with 0 Axes>